# Group 4 - Script Assignment 6 

## Problem Statement

In Financial Technology (FinTech), anomaly detection plays a crucial role in identifying fraudulent activities such as money laundering or unauthorized transactions. One popular method for anomaly detection is DBSCAN (Density-Based Spatial Clustering of Applications with Noise), which effectively identifies clusters of data points that are closely packed together while marking outliers as noise. In this assignment, you are tasked with implementing a Python script to detect anomalous transactions in financial data using DBSCAN.

## Download the provided financial transaction datasetLinks to an external site. Preprocess the data if necessary (e.g., normalization, feature engineering). (5 points)

### Imports and Read File

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

# Load transaction data from the chosen file
transactions = pd.read_excel('../data/bank.xlsx')
transactions

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1.000000e+06,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2.000000e+06,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2.500000e+06,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5.500000e+06,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6.000000e+06,.
...,...,...,...,...,...,...,...,...,...
116196,409000362497',2019-03-05,TRF TO 1196428 Indiaforensic SE,NaN,2019-03-05,117934.30,NaN,-1.901902e+09,.
116197,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901602e+09,.
116198,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901302e+09,.
116199,409000362497',2019-03-05,IMPS 05-03-20194C,NaN,2019-03-05,109868.65,NaN,-1.901412e+09,.


### Review dataset

In [28]:
transactions.describe()

,DATE,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT
count,116201,905.000000,116201,5.354900e+04,6.265200e+04,1.162010e+05
mean,2017-05-20 00:08:40.477448448,791614.503867,2017-05-20 00:04:43.288439808,4.489190e+06,3.806586e+06,-1.404852e+09
min,2015-01-01 00:00:00,1.000000,2015-01-01 00:00:00,1.000000e-02,1.000000e-02,-2.045201e+09
25%,2016-05-30 00:00:00,704231.000000,2016-05-30 00:00:00,3.000000e+03,9.900000e+04,-1.690383e+09
50%,2017-06-05 00:00:00,873812.000000,2017-06-05 00:00:00,4.708300e+04,4.265000e+05,-1.661395e+09
75%,2018-05-26 00:00:00,874167.000000,2018-05-26 00:00:00,5.000000e+06,4.746411e+06,-1.236888e+09
max,2019-03-05 00:00:00,874525.000000,2019-03-05 00:00:00,4.594475e+08,5.448000e+08,8.500000e+06
std,NaN,151205.932910,NaN,1.084850e+07,8.683093e+06,5.348202e+08


In [29]:
transactions.isnull().sum()

Account No                  0
DATE                        0
TRANSACTION DETAILS      2499
CHQ.NO.                115296
VALUE DATE                  0
WITHDRAWAL AMT          62652
DEPOSIT AMT             53549
BALANCE AMT                 0
.                           0
dtype: int64

### Clean and drop features

In [30]:
# Replace missing values with 0 for WITHDRAWAL AMT and DEPOSIT AMT

transactions['WITHDRAWAL AMT'] = transactions['WITHDRAWAL AMT'].fillna(0)
transactions['DEPOSIT AMT'] = transactions['DEPOSIT AMT'].fillna(0)


# Remove trailing single quote from Account No
transactions['Account No'] = transactions['Account No'].str.replace("'", "")

In [31]:
# Standardize the data
scaler = StandardScaler()
transactions_scaled = scaler.fit_transform(transactions)
transactions_scaled.head(20)

TypeError: float() argument must be a string or a real number, not 'Timestamp'

In [ ]:

# DBSCAN clustering
eps_values = [0.1, 0.5, 1.0]  
min_samples_values = [5, 10, 15]  

best_score = -1
best_params = None
best_labels = None

# Number of clusters, ignoring noise
for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(transactions_scaled)
        score = len(set(labels)) - (1 if -1 in labels else 0)  
        if score > best_score:
            best_score = score
            best_params = (eps, min_samples)
            best_labels = labels

# Extracting the best parameters and clustering results
best_eps, best_min_samples = best_params
print("Best parameters: eps={}, min_samples={}".format(best_eps, best_min_samples))
print("Number of clusters found:", best_score)

# Assigning cluster labels to the original data
transactions['cluster'] = best_labels

# Accessing clustered transactions
for cluster_id in transactions['cluster'].unique():
    cluster_transactions = transactions[transactions['cluster'] == cluster_id]
    print(f"Cluster {cluster_id}:")
    print(cluster_transactions.head())

# Analyzing anomalies
anomaly_mask = best_labels == -1  
anomalies = transactions[anomaly_mask]

# Analyzing characteristics of anomalies
anomalies_description = anomalies.describe()
print("Characteristics of anomalies:")
print(anomalies_description)

# Visualizing clusters and outliers
plt.figure(figsize=(10, 6))

# Plotting clustered transactions
plt.scatter(transactions['BALANCE AMT'], transactions['DATE'], c=labels, cmap='viridis', alpha=0.5)
plt.colorbar(label='Cluster')
plt.title('DBSCAN Clustering of Transactions')
plt.xlabel('BALANCE AMT')
plt.ylabel('DATE')
plt.grid(True)

# Highlighting anomalies
plt.scatter(anomalies['BALANCE AMT'], anomalies['DATE'], color='red', label='Anomalies')
plt.legend()

plt.show()